<a href="https://colab.research.google.com/github/mightyoctopus/lora-fine-tuning-example-code/blob/main/LoRA_PG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -q peft transformers datasets torch accelerate

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 40.2 MB/s eta 0:00:00


Dataset: Rabe3/QA_Synthatic_Medical_data

Model (Instrcut): Qwen/Qwen3-0.6B

- quantization
- base model load and load peft model (with LoraConfig)
- tokenization (batches)
- load dataset
- map dataset to be tokenized
- Train (with TrainingArguments configued)


In [3]:
from torch.utils.data import DataLoader
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
                          )
from peft import LoraConfig, get_peft_model, TaskType

from google.colab import drive, userdata
from huggingface_hub import login, snapshot_download

import os

In [4]:
model_name = "Qwen/Qwen3-0.6B"
dataset_name = "Rabe3/QA_Synthatic_Medical_data"

In [5]:
drive.mount("/content/drive")
cache_path = "/content/drive/My Drive/models/huggingface_cache"

Mounted at /content/drive


In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
model_folder_name = "models--" + model_name.replace("/", "--")
parent_model_path_in_drive = os.path.join(cache_path, model_folder_name)

if not os.path.exists(parent_model_path_in_drive):
    print("Model not found in Drive -- downloading from HF...")
    model_path = snapshot_download(
        repo_id=model_name,
        cache_dir=cache_path,
        local_dir_use_symlinks=False
    )
else:
    print("Model found in Drive -- fetching from the cache...")
    snapshots_dir = os.path.join(parent_model_path_in_drive, "snapshots")
    drive_id = os.listdir(snapshots_dir)

    if drive_id:
        model_path = os.path.join(snapshots_dir, drive_id[0])
    else:
        raise ValueError("No snapshot found in the cache path.")

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    cache_dir=cache_path
)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)


Model found in Drive -- fetching from the cache...


In [7]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha=16,
    bias="none",
    lora_dropout=0.05,
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [8]:
data = load_dataset(dataset_name, "default", split="train[:200]")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.jsonl:   0%|          | 0.00/45.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20750 [00:00<?, ? examples/s]

In [9]:
def tokenize(batch):
    texts = []

    for convo in batch["conversations"]:
        for turn in convo:
            human_msg = turn["value"] if turn["from"] == "human" else ""
            assisant_msg = turn["value"] if turn["from"] == "gpt" else ""

            texts.append(f"### Instruction:\n{human_msg}\n### Response:\n{assisant_msg}")


    tokens = tokenizer(
        texts,
        padding="max_length",
        max_length=256,
        truncation=True,
        return_tensors="pt"
    )

    print("TOKENS", tokens)

    tokens["labels"] = tokens["input_ids"].clone()

    return tokens

In [10]:
tokenized_data = data.map(tokenize, batched=True, remove_columns=data.column_names)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

TOKENS {'input_ids': tensor([[ 14374,  29051,    510,  ..., 151643, 151643, 151643],
        [ 14374,  29051,   1447,  ..., 151643, 151643, 151643],
        [ 14374,  29051,    510,  ..., 151643, 151643, 151643],
        ...,
        [ 14374,  29051,   1447,  ..., 151643, 151643, 151643],
        [ 14374,  29051,    510,  ..., 151643, 151643, 151643],
        [ 14374,  29051,   1447,  ...,  36671,   1142,    367]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])}


In [11]:
training_args = TrainingArguments(
    output_dir = "./fine_tuned_result",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate= 1e-3,
    num_train_epochs=50,
    fp16=True,
    logging_steps=20,
    save_strategy="epoch",
    remove_unused_columns=False,
    label_names=["labels"],
    report_to="none"
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    processing_class=tokenizer
)

In [13]:
result = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
20,2.663500
40,1.217800
60,1.084800
80,1.058500
100,0.949500
120,0.893200
140,0.746000
160,0.714200
180,0.617700
200,0.565500


In [14]:
print(result.training_loss)

0.2442795099258423


In [15]:
### Get Perplexity:
import math

loss = result.training_loss
perplexity = math.exp(loss)

print(f"Perplexity: {perplexity}")

Perplexity: 1.2767011312609746


In [16]:
### Log in to Hugging Face to push the model
user_token = userdata.get("HF_TOKEN")
login(user_token)


model.push_to_hub("MightyOctopus/qwen3-0.6B-lora-medical")
tokenizer.push_to_hub("MightyOctopus/qwen3-0.6B-lora-medical")


README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:  12%|#1        |  551kB / 4.60MB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mp3d6dsh25/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/MightyOctopus/qwen3-0.6B-lora-medical/commit/28d01a833606920f853621907dc5cff18a137081', commit_message='Upload tokenizer', commit_description='', oid='28d01a833606920f853621907dc5cff18a137081', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MightyOctopus/qwen3-0.6B-lora-medical', endpoint='https://huggingface.co', repo_type='model', repo_id='MightyOctopus/qwen3-0.6B-lora-medical'), pr_revision=None, pr_num=None)